<a href="https://colab.research.google.com/github/alizawba3a/fraud_detection_project/blob/main/fraud_detection_project_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===== CELL 1: Versions, imports, and constants =====

# --- Versions & Imports ---
import os, sys, warnings, math
from datetime import datetime
from functools import lru_cache

import numpy as np
import pandas as pd

# plotting & dashboard
import plotly.express as px
import plotly.graph_objects as go

# JupyterDash (works in Colab). If not installed, we'll install it below.
try:
    from jupyter_dash import JupyterDash
    import dash
    from dash import html, dcc, Input, Output, State
except Exception:
    JUPYTER_DASH_AVAILABLE = False
else:
    JUPYTER_DASH_AVAILABLE = True

warnings.filterwarnings("ignore")
SEED = 42
np.random.seed(SEED)

print("Python:", sys.version.split()[0])
print("pandas:", pd.__version__)
import plotly; print("plotly:", plotly.__version__)

# Constants & file locations
CRASHES_URL = "https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=download"
PERSONS_URL = "https://data.cityofnewyork.us/api/views/f55k-p6yu/rows.csv?accessType=download"
DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)

# NYC sanity bounds for lat/lon
NYC_BOUNDS = {'lat_min': 40.45, 'lat_max': 41.10, 'lon_min': -74.30, 'lon_max': -73.65}

# Display preferences
pd.options.display.max_columns = 200
pd.options.display.width = 180

# Safety and performance defaults
NROWS = 50_000   # Set to None to load full dataset (be careful in Colab with RAM)
MAX_MAP_POINTS = 20000  # sample limit for map plotting


Python: 3.12.12
pandas: 2.2.2
plotly: 5.24.1


In [ ]:
# ===== CELL 2: helper functions (parsing, cleaning utilities) =====

# --- Helper functions ---

def try_read_csv(url, nrows=None):
    """Robust CSV loader with retries and memory-safe options."""
    try:
        return pd.read_csv(url, low_memory=False, nrows=nrows)
    except Exception as e:
        # fallback: try smaller chunk
        print(f"Warning: failed to read at once ({e}). Trying small nrows fallback.")
        try:
            return pd.read_csv(url, low_memory=True, nrows=nrows)
        except Exception as e2:
            raise RuntimeError(f"Failed to load CSV from {url}: {e2}")

def try_parse_datetime(df, date_col_hint="CRASH DATE", time_col_hint="CRASH TIME"):
    """Find best date/time columns and return (date_col, time_col, datetime_series)."""
    # search for exact match, then contains 'DATE'
    dcol = next((c for c in df.columns if c.upper().strip() == date_col_hint), None)
    if dcol is None:
        dcol = next((c for c in df.columns if "DATE" in c.upper()), None)
    tcol = next((c for c in df.columns if c.upper().strip() == time_col_hint), None)
    if dcol is None:
        return None, None, None
    try:
        if tcol is not None:
            dt = pd.to_datetime(df[dcol].astype(str).str.strip() + " " + df[tcol].astype(str).str.strip(),
                                errors="coerce")
        else:
            dt = pd.to_datetime(df[dcol], errors="coerce")
        return dcol, tcol, dt
    except Exception:
        return dcol, tcol, pd.to_datetime(df[dcol], errors="coerce")

def standardize_borough(series):
    mapping = {
        "BKLYN": "BROOKLYN", "KINGS": "BROOKLYN", "KINGS COUNTY": "BROOKLYN",
        "NEW YORK": "MANHATTAN", "RICHMOND": "STATEN ISLAND", "STATEN": "STATEN ISLAND", "SI": "STATEN ISLAND"
    }
    def norm(x):
        if pd.isna(x): return x
        s = str(x).strip().upper()
        return mapping.get(s, s)
    return series.apply(norm)

def clip_latlon(lat, lon, bounds):
    lat = pd.to_numeric(lat, errors="coerce")
    lon = pd.to_numeric(lon, errors="coerce")
    lat_ok = lat.between(bounds['lat_min'], bounds['lat_max'])
    lon_ok = lon.between(bounds['lon_min'], bounds['lon_max'])
    return lat.where(lat_ok), lon.where(lon_ok)

def clean_vehicle_value(x):
    if pd.isna(x): return None
    s = str(x).upper().strip()
    if s in {"UNKNOWN","UNSPECIFIED","OTHER","","N/A","NA"}:
        return None
    if "TAXI" in s or "CAB" in s:
        return "TAXI"
    if any(k in s for k in ["AMBULANCE","FIRE","POLICE","EMERGENCY"]):
        return "EMERGENCY"
    if "BUS" in s:
        return "BUS"
    if any(k in s for k in ["MOTOR","MOPED"]) and "E-" not in s and "E " not in s:
        return "MOTORCYCLE"
    if any(k in s for k in ["BICYCLE","E-BIKE","E BIKE"]):
        return "BICYCLE"
    if "SCOOT" in s:
        return "SCOOTER"
    if any(k in s for k in ["SUV","SPORT UTILITY","PICK","STATION WAGON"]):
        return "SUV/PICKUP"
    if any(k in s for k in ["TRUCK","TRACTOR","CEMENT","FREIGHT","DELIVERY","BOX"]):
        return "TRUCK/COMMERCIAL"
    if any(k in s for k in ["SEDAN","CAR","COUPE","HATCHBACK","PASSENGER"]):
        return "PASSENGER CAR"
    return "OTHER"

def clean_factor_value(x):
    if pd.isna(x): return None
    s = str(x).upper().strip()
    if s in {"UNSPECIFIED","UNKNOWN","","N/A","NA"}:
        return None
    if s.isdigit():
        return None
    return s


In [ ]:
# ===== CELL 3: Load datasets (safe) =====

print("Loading datasets (NROWS={}):".format(NROWS))
df_crashes = try_read_csv(CRASHES_URL, nrows=NROWS)
df_persons = try_read_csv(PERSONS_URL, nrows=NROWS)

print("Crashes shape:", df_crashes.shape)
print("Persons shape:", df_persons.shape)

# Quick preview (first 3 rows)
display(df_crashes.head(3))
display(df_persons.head(3))


Loading datasets (NROWS=50000):
Crashes shape: (50000, 29)
Persons shape: (50000, 21)


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2,0,0,0,0,0,2,0,Aggressive Driving/Road Rage,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1,0,0,0,0,0,1,0,Pavement Slippery,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,11/01/2023,1:29,BROOKLYN,11230.0,40.62179,-73.970024,"(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,NaN,1,0,0,0,0,0,1,0,Unspecified,Unspecified,Unspecified,NaN,NaN,4675373,Moped,Sedan,Sedan,NaN,NaN


,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,PERSON_ID,PERSON_TYPE,PERSON_INJURY,VEHICLE_ID,PERSON_AGE,EJECTION,EMOTIONAL_STATUS,BODILY_INJURY,POSITION_IN_VEHICLE,SAFETY_EQUIPMENT,PED_LOCATION,PED_ACTION,COMPLAINT,PED_ROLE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2,PERSON_SEX
0,10249006,4229554,10/26/2019,9:43,31aa2bc0-f545-444f-8cdb-f1cb5cf00b89,Occupant,Unspecified,19141108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Registrant,NaN,NaN,U
1,10255054,4230587,10/25/2019,15:15,4629e500-a73e-48dc-b8fb-53124d124b80,Occupant,Unspecified,19144075.0,33.0,Not Ejected,Does Not Apply,Does Not Apply,"Front passenger, if two or more persons, inclu...",Lap Belt & Harness,NaN,NaN,Does Not Apply,Passenger,NaN,NaN,F
2,10253177,4230550,10/26/2019,17:55,ae48c136-1383-45db-83f4-2a5eecfb7cff,Occupant,Unspecified,19143133.0,55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Registrant,NaN,NaN,M


In [ ]:
# ===== CELL 4: Pre-integration cleaning for crashes =====

# 1) Missingness summary (top 20)
def missing_report(df, top_n=20):
    mis = df.isna().mean().sort_values(ascending=False) * 100
    return mis.head(top_n).to_frame("missing_%")

print("Top missingness (crashes):")
display(missing_report(df_crashes, 20))

# 2) Parse date/time robustly
crash_date_col, crash_time_col, crash_dt = try_parse_datetime(df_crashes)
if crash_dt is not None:
    df_crashes["CRASH_DATETIME"] = crash_dt
    df_crashes["CRASH_YEAR"] = df_crashes["CRASH_DATETIME"].dt.year
    df_crashes["CRASH_MONTH"] = df_crashes["CRASH_DATETIME"].dt.month
    df_crashes["CRASH_DOW"] = df_crashes["CRASH_DATETIME"].dt.day_name()
    df_crashes["CRASH_HOUR"] = df_crashes["CRASH_DATETIME"].dt.hour
    print("Parsed CRASH_DATETIME from", crash_date_col, "and", crash_time_col)

# 3) Standardize borough column
borough_col = next((c for c in df_crashes.columns if "BOROUGH" in c.upper()), None)
if borough_col:
    df_crashes[borough_col] = standardize_borough(df_crashes[borough_col])

# 4) Lat/lon sanity
lat_col = next((c for c in df_crashes.columns if "LATITUDE" in c.upper()), None)
lon_col = next((c for c in df_crashes.columns if "LONGITUDE" in c.upper()), None)
if lat_col and lon_col:
    df_crashes[lat_col] = pd.to_numeric(df_crashes[lat_col], errors="coerce")
    df_crashes[lon_col] = pd.to_numeric(df_crashes[lon_col], errors="coerce")
    df_crashes[lat_col], df_crashes[lon_col] = clip_latlon(df_crashes[lat_col], df_crashes[lon_col], NYC_BOUNDS)
    print("Applied lat/lon clipping to", lat_col, lon_col)

# 5) Remove exact duplicates
before = len(df_crashes)
df_crashes = df_crashes.drop_duplicates()
print(f"Removed {before - len(df_crashes):,} exact duplicate rows")

# 6) Clip negative injury/fatal counts
inj_fatal_cols = [c for c in df_crashes.columns if any(k in c.upper() for k in ["INJUR","FATAL"])]
for c in inj_fatal_cols:
    if pd.api.types.is_numeric_dtype(df_crashes[c]):
        neg = (df_crashes[c] < 0).sum()
        if neg:
            df_crashes[c] = df_crashes[c].clip(lower=0)
            print(f"Clipped {neg} negatives in {c}")

# 7) Drop columns with >60% missing (safe)
missing_pct = df_crashes.isna().mean()
drop_cols = missing_pct[missing_pct > 0.6].index.tolist()
if drop_cols:
    df_crashes.drop(columns=drop_cols, inplace=True)
    print("Dropped columns with >60% missing:", drop_cols)

# 8) Vehicle & factor cleaning across potential vehicle columns
vehicle_cols = [c for c in df_crashes.columns if "VEHICLE TYPE" in c.upper()]
for col in vehicle_cols:
    df_crashes[col] = df_crashes[col].apply(clean_vehicle_value)

factor_cols = [c for c in df_crashes.columns if "CONTRIBUTING FACTOR" in c.upper()]
for col in factor_cols:
    df_crashes[col] = df_crashes[col].apply(clean_factor_value)

print("After pre-integration cleaning:", df_crashes.shape)


Top missingness (crashes):


,missing_%
VEHICLE TYPE CODE 5,99.186
CONTRIBUTING FACTOR VEHICLE 5,99.154
VEHICLE TYPE CODE 4,97.248
CONTRIBUTING FACTOR VEHICLE 4,97.100
VEHICLE TYPE CODE 3,90.110
CONTRIBUTING FACTOR VEHICLE 3,89.338
OFF STREET NAME,72.854
CROSS STREET NAME,53.826
ZIP CODE,34.594
BOROUGH,34.582


Parsed CRASH_DATETIME from CRASH DATE and CRASH TIME
Applied lat/lon clipping to LATITUDE LONGITUDE
Removed 0 exact duplicate rows
Dropped columns with >60% missing: ['OFF STREET NAME', 'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']
After pre-integration cleaning: (50000, 27)


In [ ]:
# ===== CELL 5: Clean & aggregate persons dataset, then integrate =====

# Make COLLISION_ID numeric (present in both tables)
if "COLLISION_ID" in df_persons.columns:
    df_persons["COLLISION_ID"] = pd.to_numeric(df_persons["COLLISION_ID"], errors="coerce")

# detect person type and injury/fatal columns in persons
person_type_col = next((c for c in df_persons.columns if "PERSON_TYPE" in c.upper()), None)
inj_col = next((c for c in df_persons.columns if "INJUR" in c.upper()), None)
fatal_col = next((c for c in df_persons.columns if "FATAL" in c.upper()), None)

# Create aggregation from persons: counts per person type + sums of injury/fatal if available
agg = None
if person_type_col:
    pt_counts = (df_persons.groupby(["COLLISION_ID", person_type_col])
                 .size().unstack(fill_value=0))
    pt_counts.columns = ["PERSON_CT__" + str(c).upper().replace(" ", "_") for c in pt_counts.columns]
    agg = pt_counts

add_cols = []
if inj_col and inj_col in df_persons.columns: add_cols.append(inj_col)
if fatal_col and fatal_col in df_persons.columns: add_cols.append(fatal_col)
if add_cols:
    sums = df_persons.groupby("COLLISION_ID")[add_cols].sum(min_count=1)
    agg = sums if agg is None else agg.join(sums, how="outer")

# merge
key_col = next((c for c in df_crashes.columns if c.upper() == "COLLISION_ID"), None)
if agg is not None:
    agg = agg.reset_index()
    df_integrated = df_crashes.merge(agg, left_on=key_col, right_on="COLLISION_ID", how="left", validate="one_to_one")
else:
    # fallback: join with persons (may be many-to-one)
    if "COLLISION_ID" in df_persons.columns:
        df_integrated = df_crashes.merge(df_persons, on="COLLISION_ID", how="left", suffixes=("", "_PERSON"))
    else:
        df_integrated = df_crashes.copy()

print("Integrated shape:", df_integrated.shape)
display(df_integrated.head(3))


Integrated shape: (50000, 31)


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,CRASH_DATETIME,CRASH_YEAR,CRASH_MONTH,CRASH_DOW,CRASH_HOUR,PERSON_CT__BICYCLIST,PERSON_CT__OCCUPANT,PERSON_CT__PEDESTRIAN,PERSON_INJURY
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,2,0,0,0,0,0,2,0,AGGRESSIVE DRIVING/ROAD RAGE,None,4455765,PASSENGER CAR,PASSENGER CAR,2021-09-11 02:39:00,2021,9,Saturday,2,NaN,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,1,0,0,0,0,0,1,0,PAVEMENT SLIPPERY,None,4513547,PASSENGER CAR,None,2022-03-26 11:45:00,2022,3,Saturday,11,NaN,NaN,NaN,NaN
2,11/01/2023,1:29,BROOKLYN,11230.0,40.62179,-73.970024,"(40.62179, -73.970024)",OCEAN PARKWAY,AVENUE K,1,0,0,0,0,0,1,0,None,None,4675373,MOTORCYCLE,PASSENGER CAR,2023-11-01 01:29:00,2023,11,Wednesday,1,NaN,NaN,NaN,NaN


In [ ]:
# ===== CELL 6: Post-integration cleaning & dtype harmonization =====

# 1) Convert injury/fatal/person count columns to nullable Int (Int64) where sensible
for col in df_integrated.columns:
    if any(k in col.upper() for k in ["COUNT", "CT__", "INJUR", "FATAL"]):
        try:
            df_integrated[col] = pd.to_numeric(df_integrated[col], errors="coerce").astype("Int64")
        except Exception:
            # keep original if conversion fails
            pass

# 2) Drop all-empty columns
empty_cols = [c for c in df_integrated.columns if df_integrated[c].isna().all()]
if empty_cols:
    df_integrated = df_integrated.drop(columns=empty_cols)
    print("Dropped all-NA columns:", empty_cols)

# 3) Fill zeros for injury/fatal/person ct columns to avoid NaNs
fill_zero_cols = [c for c in df_integrated.columns if any(k in c.upper() for k in ["INJUR","FATAL","PERSON_CT__"])]
for c in fill_zero_cols:
    if c in df_integrated.columns:
        df_integrated[c] = df_integrated[c].fillna(0)

# 4) Drop duplicate or redundant columns from merge suffixes
dup_cols = [c for c in df_integrated.columns if c.endswith("_y") or c.endswith("_PERSON")]
if dup_cols:
    df_integrated = df_integrated.drop(columns=dup_cols)
    print("Dropped redundant columns:", dup_cols)

# 5) Drop rows missing essential columns (datetime, lat, lon)
essential_cols = ["CRASH_DATETIME", lat_col, lon_col]
essential_cols = [c for c in essential_cols if c is not None]
if essential_cols:
    drop_rows = df_integrated[essential_cols].isna().any(axis=1).sum()
    if drop_rows > 0:
        df_integrated = df_integrated.dropna(subset=essential_cols)
        print(f"Dropped {drop_rows} rows missing essential info: {essential_cols}")

print("Post-integration shape:", df_integrated.shape)
df_integrated.info(memory_usage="deep")


Dropped all-NA columns: ['PERSON_INJURY']
Dropped 4191 rows missing essential info: ['CRASH_DATETIME', 'LATITUDE', 'LONGITUDE']
Post-integration shape: (45809, 30)
<class 'pandas.core.frame.DataFrame'>
Index: 45809 entries, 2 to 49999
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   CRASH DATE                     45809 non-null  object        
 1   CRASH TIME                     45809 non-null  object        
 2   BOROUGH                        31689 non-null  object        
 3   ZIP CODE                       31685 non-null  float64       
 4   LATITUDE                       45809 non-null  float64       
 5   LONGITUDE                      45809 non-null  float64       
 6   LOCATION                       45809 non-null  object        
 7   ON STREET NAME                 32734 non-null  object        
 8   CROSS STREET NAME              21192 non-null  object    

In [ ]:
# ===== CELL 7: Exploratory Data Analysis (plots & small checks) =====

# Summary statistics
display(df_integrated.describe(include="all"))

# 1) Crashes per Year bar (safe: dropna)
if "CRASH_YEAR" in df_integrated.columns:
    year_counts = df_integrated["CRASH_YEAR"].dropna().value_counts().sort_index().reset_index()
    year_counts.columns = ["CRASH_YEAR","count"]
    fig_year = px.bar(year_counts, x="CRASH_YEAR", y="count", title="Crashes per Year")
    fig_year.show()

# 2) Crashes by Borough
if borough_col and borough_col in df_integrated.columns:
    borough_counts = df_integrated[borough_col].value_counts(dropna=False).reset_index()
    borough_counts.columns = [borough_col, "count"]
    fig_b = px.bar(borough_counts, x=borough_col, y="count", title="Crashes by Borough")
    fig_b.show()

# 3) Monthly trend by year (line)
if {"CRASH_YEAR","CRASH_MONTH"}.issubset(df_integrated.columns):
    monthly = (df_integrated.groupby(["CRASH_YEAR","CRASH_MONTH"]).size().reset_index(name="count"))
    monthly["Month"] = pd.to_datetime(monthly["CRASH_MONTH"], format="%m", errors="coerce").dt.strftime("%b")
    fig_month = px.line(monthly, x="Month", y="count", color="CRASH_YEAR", title="Monthly Crash Trend by Year")
    fig_month.show()

# 4) Injuries/Fatalities by Year (sums)
inj_fatal_cols = [c for c in df_integrated.columns if any(k in c.upper() for k in ["INJUR","FATAL"]) ]
if "CRASH_YEAR" in df_integrated.columns and inj_fatal_cols:
    yearly_sums = df_integrated.groupby("CRASH_YEAR")[inj_fatal_cols].sum(min_count=1).reset_index()
    fig_if = px.line(yearly_sums, x="CRASH_YEAR", y=inj_fatal_cols, title="Yearly Injuries & Fatalities", markers=True)
    fig_if.show()

# 5) Hour vs Day heatmap (sample safe)
if {"CRASH_HOUR","CRASH_DOW"}.issubset(df_integrated.columns):
    piv = (df_integrated.groupby(["CRASH_DOW","CRASH_HOUR"]).size().reset_index(name="count"))
    order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    piv["CRASH_DOW"] = pd.Categorical(piv["CRASH_DOW"], order, ordered=True)
    fig_heat = px.density_heatmap(piv, x="CRASH_HOUR", y="CRASH_DOW", z="count", nbinsx=24, title="Crashes by Hour & Day")
    fig_heat.show()

# 6) Map of sample crash locations
if lat_col and lon_col and lat_col in df_integrated.columns and lon_col in df_integrated.columns:
    tmp = df_integrated.dropna(subset=[lat_col, lon_col])
    if len(tmp) > 0:
        sample_n = min(MAX_MAP_POINTS, len(tmp))
        sample = tmp.sample(sample_n, random_state=SEED)
        fig_map = px.scatter_geo(sample, lat=lat_col, lon=lon_col, scope="usa", title="Sample Crash Locations", opacity=0.5)
        fig_map.update_geos(fitbounds="locations", visible=False)
        fig_map.show()

# Save cleaned integrated dataset for dashboard / further work
out_path = os.path.join(DATA_DIR, "cleaned_integrated_nyc_crashes.csv")
df_integrated.to_csv(out_path, index=False)
print("Saved cleaned integrated dataset to", out_path)


,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,CRASH_DATETIME,CRASH_YEAR,CRASH_MONTH,CRASH_DOW,CRASH_HOUR,PERSON_CT__BICYCLIST,PERSON_CT__OCCUPANT,PERSON_CT__PEDESTRIAN
count,45809,45809,31689,31685.000000,45809.000000,45809.000000,45809,32734,21192,45809.0,45809.000000,45809.0,45809.000000,45809.0,45809.000000,45809.0,45809.000000,34158,4995,4.580900e+04,45242,30562,45809,45809.000000,45809.000000,45809,45809.000000,45809.0,45809.0,45809.0
unique,582,1440,5,NaN,NaN,NaN,29374,3018,3394,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,54,41,NaN,10,10,NaN,NaN,NaN,7,NaN,<NA>,<NA>,<NA>
top,05/27/2021,0:00,BROOKLYN,NaN,NaN,NaN,"(40.675735, -73.89686)",BELT PARKWAY,3 AVENUE,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,DRIVER INATTENTION/DISTRACTION,DRIVER INATTENTION/DISTRACTION,NaN,PASSENGER CAR,PASSENGER CAR,NaN,NaN,NaN,Friday,NaN,<NA>,<NA>,<NA>
freq,378,775,11145,NaN,NaN,NaN,35,781,213,<NA>,NaN,<NA>,NaN,<NA>,NaN,<NA>,NaN,11169,1848,NaN,21908,13415,NaN,NaN,NaN,7031,NaN,<NA>,<NA>,<NA>
mean,NaN,NaN,NaN,10896.953985,40.724006,-73.916176,NaN,NaN,NaN,0.467899,0.002751,0.072453,0.001113,0.047938,0.000196,0.331638,0.001288,NaN,NaN,4.455108e+06,NaN,NaN,2021-09-09 08:25:32.697505024,2021.192451,6.464625,NaN,12.841429,0.0,0.000044,0.0
min,NaN,NaN,NaN,10000.000000,40.499710,-74.251840,NaN,NaN,NaN,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,NaN,NaN,3.456194e+06,NaN,NaN,2012-09-25 12:36:00,2012.000000,1.000000,NaN,0.000000,0.0,0.0,0.0
25%,NaN,NaN,NaN,10457.000000,40.666134,-73.962390,NaN,NaN,NaN,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,NaN,NaN,4.420057e+06,NaN,NaN,2021-05-22 08:40:00,2021.000000,5.000000,NaN,8.000000,0.0,0.0,0.0
50%,NaN,NaN,NaN,11208.000000,40.713398,-73.920280,NaN,NaN,NaN,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,NaN,NaN,4.432908e+06,NaN,NaN,2021-06-29 11:00:00,2021.000000,6.000000,NaN,14.000000,0.0,0.0,0.0
75%,NaN,NaN,NaN,11238.000000,40.785030,-73.868650,NaN,NaN,NaN,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,NaN,NaN,4.488503e+06,NaN,NaN,2021-12-18 20:09:00,2021.000000,9.000000,NaN,18.000000,0.0,0.0,0.0
max,NaN,NaN,NaN,11697.000000,40.912827,-73.700710,NaN,NaN,NaN,18.0,3.000000,6.0,1.000000,3.0,1.000000,18.0,3.000000,NaN,NaN,4.763285e+06,NaN,NaN,2024-10-09 09:58:00,2024.000000,12.000000,NaN,23.000000,0.0,1.0,0.0


Saved cleaned integrated dataset to data/cleaned_integrated_nyc_crashes.csv


In [ ]:
# ===== CELL 8: Lightweight Dash app (JupyterDash) with Generate Report button =====

# This app provides:
# - Dropdowns: Borough, Year, Vehicle Type, Contributing Factor
# - Search input that attempts to parse simple queries
# - A Generate Report button that updates multiple figures

# If jupyter_dash not available, the cell prints instructions to install it.
if not JUPYTER_DASH_AVAILABLE:
    print("JupyterDash (jupyter_dash) not available. Install it with: !pip install jupyter-dash dash")
else:
    # Prepare dropdown options (safe unique lists)
    def safe_unique(col):
        if col is None or col not in df_integrated.columns:
            return []
        vals = df_integrated[col].dropna().unique().tolist()
        vals_sorted = sorted([v for v in vals if v is not None])
        return vals_sorted

    borough_opts = safe_unique(borough_col)
    year_opts = sorted([int(y) for y in df_integrated["CRASH_YEAR"].dropna().unique().tolist()]) if "CRASH_YEAR" in df_integrated.columns else []
    # aggregate vehicle types across vehicle columns
    vehicle_cols = [c for c in df_integrated.columns if "VEHICLE TYPE" in c.upper()]
    vehicle_set = set()
    for vc in vehicle_cols:
        vehicle_set.update([v for v in df_integrated[vc].dropna().unique() if v is not None])
    vehicle_opts = sorted(vehicle_set)
    factor_opts = set()
    for fc in factor_cols:
        factor_opts.update([v for v in df_integrated[fc].dropna().unique() if v is not None])
    factor_opts = sorted(factor_opts)

    # Build Dash app
    app = JupyterDash(__name__)
    app.layout = html.Div([
        html.H3("NYC Crashes Explorer (Milestone 1)"),
        html.Div([
            html.Div([
                html.Label("Borough"),
                dcc.Dropdown(options=[{"label":b,"value":b} for b in borough_opts], multi=True, id="borough-filter")
            ], style={"width":"24%","display":"inline-block","verticalAlign":"top"}),
            html.Div([
                html.Label("Year"),
                dcc.Dropdown(options=[{"label":str(y),"value":y} for y in year_opts], multi=True, id="year-filter")
            ], style={"width":"24%","display":"inline-block","marginLeft":"1%","verticalAlign":"top"}),
            html.Div([
                html.Label("Vehicle Type"),
                dcc.Dropdown(options=[{"label":v,"value":v} for v in vehicle_opts], multi=True, id="vehicle-filter")
            ], style={"width":"24%","display":"inline-block","marginLeft":"1%","verticalAlign":"top"}),
            html.Div([
                html.Label("Contrib. Factor"),
                dcc.Dropdown(options=[{"label":f,"value":f} for f in factor_opts], multi=True, id="factor-filter")
            ], style={"width":"24%","display":"inline-block","marginLeft":"1%","verticalAlign":"top"}),
        ], style={"marginBottom":"12px"}),
        html.Div([
            html.Label("Search (e.g., 'Brooklyn 2022 pedestrian crashes')"),
            dcc.Input(id="search-input", type="text", placeholder="type search and press Generate Report", style={"width":"80%"}),
            html.Button("Generate Report", id="generate-btn", n_clicks=0, style={"marginLeft":"8px"})
        ], style={"marginBottom":"16px"}),
        html.Div(id="summary-text", style={"marginTop":"10px","fontWeight":"bold"}),
        html.Div([
            dcc.Graph(id="graph-year"),
            dcc.Graph(id="graph-borough"),
            dcc.Graph(id="graph-heat"),
        ])
    ], style={"padding":"10px"})

    # Data filtering helper
    @lru_cache(maxsize=32)
    def filter_dataframe(boroughs_tuple, years_tuple, vehicles_tuple, factors_tuple, search_text):
        d = df_integrated
        # apply filters (tuples come from dash as tuples)
        if boroughs_tuple:
            d = d[d[borough_col].isin(boroughs_tuple)]
        if years_tuple:
            d = d[d["CRASH_YEAR"].isin(list(years_tuple))]
        if vehicles_tuple and vehicle_cols:
            # if multiple vehicle columns, check any contains the selected vehicle types
            sel = set(vehicles_tuple)
            mask = pd.Series(False, index=d.index)
            for vc in vehicle_cols:
                mask = mask | d[vc].isin(sel)
            d = d[mask]
        if factors_tuple and factor_cols:
            sel = set(factors_tuple)
            mask = pd.Series(False, index=d.index)
            for fc in factor_cols:
                mask = mask | d[fc].isin(sel)
            d = d[mask]

        # naive search parsing: split words and check in borough, vehicle type, and year tokens
        if search_text and isinstance(search_text, str) and search_text.strip():
            s = search_text.strip().upper()
            tokens = s.split()
            mask_s = pd.Series(True, index=d.index)
            # attempt to match borough tokens
            for t in tokens:
                if t.isdigit():
                    try:
                        tnum = int(t)
                        mask_s = mask_s & ((d["CRASH_YEAR"] == tnum) | (d.get("CRASH_MONTH") == tnum))
                    except Exception:
                        pass
                else:
                    # check borough, contributing factors, vehicle columns, and description-like columns
                    cond = pd.Series(False, index=d.index)
                    if borough_col:
                        cond = cond | d[borough_col].astype(str).str.upper().str.contains(t, na=False)
                    for vc in vehicle_cols:
                        cond = cond | d[vc].astype(str).str.upper().str.contains(t, na=False)
                    for fc in factor_cols:
                        cond = cond | d[fc].astype(str).str.upper().str.contains(t, na=False)
                    # also search in LOCATION, ON STREET NAME, CROSS STREET
                    text_cols = [c for c in d.columns if any(k in c.upper() for k in ["LOCATION","STREET","ON STREET","CROSS"])]
                    for tc in text_cols:
                        cond = cond | d[tc].astype(str).str.upper().str.contains(t, na=False)
                    mask_s = mask_s & cond
            d = d[mask_s]
        return d

    @app.callback(
        Output("summary-text","children"),
        Output("graph-year","figure"),
        Output("graph-borough","figure"),
        Output("graph-heat","figure"),
        Input("generate-btn","n_clicks"),
        State("borough-filter","value"),
        State("year-filter","value"),
        State("vehicle-filter","value"),
        State("factor-filter","value"),
        State("search-input","value"),
    )
    def update_all(n_clicks, boroughs, years, vehicles, factors, search_text):
        # Dash sends None for empty multi dropdowns; convert to tuples for caching
        b_t = tuple(boroughs) if boroughs else tuple()
        y_t = tuple(years) if years else tuple()
        v_t = tuple(vehicles) if vehicles else tuple()
        f_t = tuple(factors) if factors else tuple()

        d = filter_dataframe(b_t, y_t, v_t, f_t, search_text or "")

        # summary
        summary = f"Filtered records: {len(d):,} (Generated: {datetime.utcnow().strftime('%Y-%m-%d %H:%M UTC')})"

        # figure 1 - crashes per year (top years only)
        if "CRASH_YEAR" in d.columns and not d.empty:
            yc = d["CRASH_YEAR"].value_counts().sort_index().reset_index()
            yc.columns = ["CRASH_YEAR","count"]
            fig1 = px.bar(yc, x="CRASH_YEAR", y="count", title="Crashes per Year (filtered)")
        else:
            fig1 = go.Figure().add_annotation(text="No data for Year plot", showarrow=False)

        # figure 2 - borough counts
        if borough_col and borough_col in d.columns and not d.empty:
            bc = d[borough_col].value_counts().reset_index()
            bc.columns = [borough_col,"count"]
            fig2 = px.bar(bc, x=borough_col, y="count", title="Crashes by Borough (filtered)")
        else:
            fig2 = go.Figure().add_annotation(text="No borough data", showarrow=False)

        # figure 3 - hour/day heat (derived)
        if {"CRASH_HOUR","CRASH_DOW"}.issubset(d.columns) and not d.empty:
            piv = d.groupby(["CRASH_DOW","CRASH_HOUR"]).size().reset_index(name="count")
            order = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
            piv["CRASH_DOW"] = pd.Categorical(piv["CRASH_DOW"], order, ordered=True)
            fig3 = px.density_heatmap(piv, x="CRASH_HOUR", y="CRASH_DOW", z="count", nbinsx=24, title="Hour vs Day heatmap")
        else:
            fig3 = go.Figure().add_annotation(text="No hour/day data", showarrow=False)

        return summary, fig1, fig2, fig3

    # Run the app inline in the notebook
    app.run_server(mode="inline", debug=False, port=8050)


JupyterDash (jupyter_dash) not available. Install it with: !pip install jupyter-dash dash
